In [1]:
import csv
from io import StringIO
from sqlalchemy import create_engine, text
import pandas as pd
import psycopg2

In [2]:
cnxn_string = ("postgresql+psycopg2://{username}:{pswd}@{host}:{port}/{database}")
engine = create_engine(
    cnxn_string.format(
        username="postgres",
        pswd="postgres",
        host="localhost",
        port="5432",
        database="sqlda"
    )
)

In [15]:
def psql_insert_copy(table, conn, keys, data_iter):
    # Pobiera połączenie DBAPI które udostępnia kursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)
        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name
        sql = 'COPY {} ({}) FROM STDIN WITH CSV' .format(table_name, columns)
        cur.copy_expert(sql, s_buf)

In [13]:
public_trans_data = pd.read_csv('E:\\public_transportation_statistics_by_zip_code.csv')
public_trans_data.head()

,zip_code,public_transportation_pct,public_transportation_population
0,1379,3.3,13
1,1440,0.4,34
2,1505,0.9,23
3,1524,0.5,20
4,1529,1.8,32


In [16]:
public_trans_data.to_sql(
    'public_trans_data',
    engine,
    if_exists='replace',
    index=False,
    method=psql_insert_copy
)

In [ ]:
query_min_max_pct = """
SELECT 
    MIN(public_transportation_pct) AS min_pct,
    MAX(public_transportation_pct) AS max_pct
FROM 
    public_trans_data
WHERE
    public_transportation_pct IS NOT NULL
"""

min_max_pct = pd.read_sql_query(query_min_max_pct,engine)

In [20]:
test = pd.read_sql_query(query_min_max_pct,engine)
print(test)

       min_pct  max_pct
0 -666666666.0    100.0
